In [46]:
import pandas as pd
import yfinance as yf
from yahoo_fin.stock_info import *
import numpy as np
from datetime import date

In [47]:
class Stock: 

    def __init__(self, ticker, period="1mo"):
        self.ticker = ticker
        self.period = period

# Gets the adjusted close of a ticker per a certain period
    def get_prices(self):
        price_data = get_data(self.ticker, start_date="01/01/2015",end_date = datetime.date.today(), index_as_date = True, interval=self.period)
        adjclose = price_data.loc[:, 'adjclose']
        return adjclose.tolist()


    # Gets a list of the returns per period in percentage form
    def get_period_returns(self):
        adjclose_list = self.get_prices()

    # Loops through the adjusted closes and sums the ubiased daily return divided by the length of the list
        period_returns = []

        for i in range(len(adjclose_list) - 1):
            daily_return = np.log(adjclose_list[i+1]/adjclose_list[i]) * 100
            if np.isnan(daily_return) == False:
                period_returns.append(daily_return)
        
        return period_returns

    # Gets the mean return of the stock per-period
    def get_mean_return(self):
        period_returns = self.get_period_returns()

        mean_return = sum(period_returns)/len(period_returns)
        return mean_return

    # Gets the standard deviation of a certain stock
    def get_risk(self):
        period_returns = self.get_period_returns()
        return np.std(period_returns, dtype=np.float64)


s1 = Stock('tsla')
s2 = Stock('^AXJO')



# Minimum variance portoflio
The following module calculates the weightings of two stocks that give the the least risk

In [48]:
class Portfolio: 
    def __init__(self, ticker_1, ticker_2, period = "1mo"):
        self.stock_1 = Stock(ticker_1, period)    
        self.stock_2 = Stock(ticker_2, period)
    
    def get_return(self, w_1, w_2):
        exp_return = w_1 * self.stock_1.get_mean_return() + w_2 * self.stock_2.get_mean_return()
        return exp_return

    def get_risk(self, w_1, w_2):
        std_1 = self.stock_1.get_risk()
        std_2 = self.stock_2.get_risk()

        if w_1 < 0:
            w_sq_1 = -(w_1**2)
            w_sq_2 = w_2**2
        elif w_2 < 0: 
            w_sq_1 = w_1**2
            w_sq_2 = -(w_2**2)
        else: 
            w_sq_1 = w_1**2
            w_sq_2 = w_2**2
        
        variance = (w_sq_1 * (std_1**2)) + (w_sq_2 * (std_2**2)) + (2 * self.get_corr() * w_1 * w_2 * std_1 * std_2)
        risk = math.sqrt(variance)
        return risk

    # Gets the covariance of the two stocks
    def get_cov(self):
        returns1 = self.stock_1.get_period_returns()
        returns2 = self.stock_2.get_period_returns()

        # Makes the list of returns the same size
        abs_diff = abs(len(returns1) - len(returns2))
        if len(returns1) > len(returns2):
            returns1 = returns1[abs_diff:]
        else: 
            returns2 = returns2[abs_diff:]
        
        cov_matrix = np.cov(returns1,returns2)

        cov = cov_matrix[0,1]
        return cov
        

    def get_corr(self):
        corr = self.get_cov()/(self.stock_1.get_risk() * self.stock_2.get_risk())
        return corr

    def get_min_risk(self):
        std1 = self.stock_1.get_risk()
        std2 = self.stock_2.get_risk()

        numerator = (std2**2) - (self.get_corr() * std1 * std2)
        denominator = (std1**2) + (std2**2) - (2 * self.get_corr() * std1 * std2)

        stock_1_weight = numerator/denominator
        stock_2_weight = 1 - stock_1_weight

        return (stock_1_weight, stock_2_weight)
    
    # def get_optimium(self): 
    



portfolio = Portfolio('tsla', 'msft')
(a,b) = portfolio.get_min_risk()
print(a,b)
print(portfolio.get_risk(0.5,0.5))
print(portfolio.get_return(1.1,-0.1))


-0.01977192819786144 1.0197719281978614
9.473718071298261
3.2229620212634496
